In [1]:
# This notebook demonstrates the Basic clustering on data using a Standard K-Means algorithm.
# Spark provides a sophisticated machine learning API for performing a variety of machine learning tasks, from classification to regression, and clustering to deep learning.

# Creating the static dataframe on the retail data

staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("dbfs:/data/retail-data/by-day/*.csv")

In [2]:
# The schema of the above dataframe.

staticDataFrame.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: timestamp (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: double (nullable = true)
-- Country: string (nullable = true)

In [3]:
staticDataFrame.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 580538| 23084| RABBIT NIGHT LIGHT| 48|2011-12-05 08:38:00| 1.79| 14075.0|United Kingdom|
 580538| 23077| DOUGHNUT LIP GLOSS | 20|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom|
 580538| 22906|12 MESSAGE CARDS ...| 24|2011-12-05 08:38:00| 1.65| 14075.0|United Kingdom|
 580538| 21914|BLUE HARMONICA IN...| 24|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom|
 580538| 22467| GUMBALL COAT RACK| 6|2011-12-05 08:38:00| 2.55| 14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

In [4]:
# Machine Learning algorithms requires that data is represented as Numerical values. But out DF contains the values in different other formats.

# date_format is used to converts a date/timestamp/string to a value of string in the format specified in 2nd arguement.

from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)

preppedDataFrame.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
 580538| 23084| RABBIT NIGHT LIGHT| 48|2011-12-05 08:38:00| 1.79| 14075.0|United Kingdom| Monday|
 580538| 23077| DOUGHNUT LIP GLOSS | 20|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom| Monday|
 580538| 22906|12 MESSAGE CARDS ...| 24|2011-12-05 08:38:00| 1.65| 14075.0|United Kingdom| Monday|
 580538| 21914|BLUE HARMONICA IN...| 24|2011-12-05 08:38:00| 1.25| 14075.0|United Kingdom| Monday|
 580538| 22467| GUMBALL COAT RACK| 6|2011-12-05 08:38:00| 2.55| 14075.0|United Kingdom| Monday|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
only showing top 5 rows

In [5]:
# Split the data into Training and Test sets. Because this is time-series data, split by arbitrary date.

trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")

In [6]:
# Checking the counts of the split datasets.
trainDataFrame.count()
testDataFrame.count()

Out[ 8 ]: 296006

In [7]:
# converting the column day_of_week string value into numerical values using StringIndexer.
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")

# StringIndexer might represents Saturday as 6, and Monday as 1. However, with this numbering scheme, we are implicitly stating that Saturday is greater than Monday. This is obviously incorrect. 

# To fix this, we therefore need to use a OneHotEncoder to encode each of these values as their own column.

# OneHotEncoder encodes each of the value into each column, where each column corresponds to one possible value of one feature.
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")

In [8]:
# All ML algorithms in Spark take Vector type as an input, assembling the numerical values into Vector Type using VectorAssembler.

from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

# 3 Features UnitPrice, Quantity and Day_of_week_encoded assemled as Vector.

In [9]:
# Set this up into a pipeline so that any future data we need to transform can go through the exact same process

from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])

In [10]:
# Fit the TransformationPipeline to the Training DataFrame.
fittedPipeline = transformationPipeline.fit(trainDataFrame)

#  FitterPipeline can now be used to transform our training DataFrame in a consistent and repeatable way.

transformedTraining = fittedPipeline.transform(trainDataFrame)

In [11]:
# Cache the TransformedTraining Dataframe into Memory, so that we can repeatedly access it at much lower cost.
transformedTraining.cache()
transformedTraining.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|day_of_week|day_of_week_index|day_of_week_encoded| features|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
 537226| 22811|SET OF 6 T-LIGHTS...| 6|2010-12-06 08:34:00| 2.95| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[2.95,...|
 537226| 21713|CITRONELLA CANDLE...| 8|2010-12-06 08:34:00| 2.1| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[2.1,8...|
 537226| 22927|GREEN GIANT GARDE...| 2|2010-12-06 08:34:00| 5.95| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[5.95,...|
 537226| 20802|SMALL GLASS SUNDA...| 6|2010-12-06 08:34:00| 1.65| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[1.65,...|
 537226| 22052|VINTAGE CARAVAN G...| 25|2010-12-06 08:34:00| 0.42| 15987.0|United Kingdom| Monday| 2.0| (5,[2],[1.0])|(7,[0,1,4],[0.42,...|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+--------------------+
only showing top 5 rows

In [12]:
# Training the model
# In Spark, training machine learning models is a two-phase process. First, we initialize an untrained model, and then we train it.
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
  .setK(20)\
  .setSeed(1L)

kmModel = kmeans.fit(transformedTraining)

In [13]:
# Computing the cost according to metrics.
kmModel.computeCost(transformedTraining)

Out[ 17 ]: 84553739.96537484

In [14]:
# Transforming the Test Dataset with the pipeline.
transformedTest = fittedPipeline.transform(testDataFrame)

# Computing the cost on Test Dataset.
kmModel.computeCost(transformedTest)

# The computed cost is high, we can continue to improve this model, layering more preprocessing as well as performing hyperparameter tuning.

Out[ 19 ]: 517507094.72221166